<a href="https://colab.research.google.com/github/mopu3263/NLP-project_Tweets-Classificaiton/blob/main/Twitter_Kamala_Vs_Biden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Twitter comments detection: we will use tweepy, NLP for text preprocessing       and use scikit learn model to detect the tweets of a specific user**


---
# **Goals**
We will use Kamala and Biden Twitter accounts as example and will classify which tweet came from whom. The steps will be:


1.   Creat a methode using Tweepy to pull a list of tweets from twitter API
2.   Perform preprocessing with textacy library
3.   Use scikit supervised classification technique to identify the user

Some Twitter Rules
THERE ARE LIMITS TO THE AMOUNT OF TIMES YOU CAN HIT THE API PER 15 MINUTE WINDOW. BEWARE!

Here's a quick overview of what Twitter says are "the rules":
https://developer.twitter.com/en/docs/rate-limits

Tweepy installation & import tweepy, panda,nump etc

In [2]:
!pip install tweepy
import tweepy
import pandas as pd
import numpy as np
import re, datetime

Twitter is a great resource of text data for Natural Language Processing (NLP) project but required some works. The below website describes how to do step by step starting from application of developer account in Twitter

https://towardsdatascience.com/tweepy-for-beginners-24baf21f2c25

In [3]:
class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':       'L7vVkmHZyAN1aEQ6KovMF72kr',
        'consumer_secret':     'Vq9W4dLPuuET08aR8D2ofcoZ4dhn1X8S37Ui1TqUlnrY7X6PLs',
        'access_token_key':    '935888914325184512-qGIL3l6bgrWrl4wPiy06CnGEkbDWlPR',
        'access_token_secret': '5hGsH2BI4W4BI53lHze54eUPEaAQu4J98hDJCtSELg8YZ'
    }
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)

        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="HillaryClinton", #BECAUSE WHO ELSE!
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:
                try:
                    mined = {
                        'tweet_id':        item.id,
                        'handle':          item.user.name,
                        'retweet_count':   item.retweet_count,
                        'text':            item.full_text,
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item.created_at,
                    }
                    
                except:
                        mined = {
                        'tweet_id':        item.id,
                        'handle':          item.user.name,
                        'retweet_count':   item.retweet_count,
                        'text':            item.full_text,
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item.created_at,
                    }
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 

# **Instantiate** **the** **class** 

In [4]:
# Result limit == count parameter 200
miner=TweetMiner(result_limit = 200 )
biden = miner.mine_user_tweets(user="JoeBiden", max_pages=14)
kamala = miner.mine_user_tweets(user="KamalaHarris", max_pages=14)

In [5]:
for x in range(5):
    print (biden[x]['text'])
    print('---')

RT @POTUS: Making vaccines free and accessible for every American — that’s the American Rescue Plan. https://t.co/5GpH0VYWIE
---
RT @POTUS: We don’t have a second to waste when it comes to delivering the American people the relief they desperately need. I’m calling on…
---
RT @POTUS: For your family, for your neighbors, for your country — wear a mask.
---
RT @POTUS: Everyone who wants a vaccine should be able to get one free of charge. The American Rescue Plan will make that a reality.
---
RT @POTUS: Last year, Michele lost her job because of the pandemic. I recently gave her a call to hear her story and discuss how my America…
---


In [6]:
for x in range(5):
    print (kamala[x]['text'])
    print('--')

RT @VP: Today @POTUS and I did a virtual tour of the State Farm Stadium in Arizona, where thousands are getting vaccinated every day. Thank…
--
RT @POTUS: Making vaccines free and accessible for every American — that’s the American Rescue Plan. https://t.co/5GpH0VYWIE
--
RT @VP: The American Rescue Plan will get the pandemic under control and support struggling communities across our country. People need hel…
--
RT @POTUS: For your family, for your neighbors, for your country — wear a mask.
--
RT @VP: It’s been a busy few weeks. Ready for the next one. https://t.co/bMouYVjRr0
--


# **Converting tweet outputs to pandas DataFrame**

In [7]:
biden_df= pd.DataFrame(biden)
print(biden_df.shape)
biden_df.head(5)

(2800, 6)


,tweet_id,handle,retweet_count,text,mined_at,created_at
0,1358851337522470914,Joe Biden,9469,RT @POTUS: Making vaccines free and accessible...,2021-02-09 03:06:46.924098,2021-02-08 18:52:59
1,1358781635240599554,Joe Biden,11383,RT @POTUS: We don’t have a second to waste whe...,2021-02-09 03:06:46.924114,2021-02-08 14:16:01
2,1358538143318880258,Joe Biden,34689,"RT @POTUS: For your family, for your neighbors...",2021-02-09 03:06:46.924117,2021-02-07 22:08:28
3,1358465618501136384,Joe Biden,16619,RT @POTUS: Everyone who wants a vaccine should...,2021-02-09 03:06:46.924119,2021-02-07 17:20:17
4,1358084646685319170,Joe Biden,8848,"RT @POTUS: Last year, Michele lost her job bec...",2021-02-09 03:06:46.924121,2021-02-06 16:06:26


In [8]:
kamala_df= pd.DataFrame(kamala)
print(kamala_df.shape)
kamala_df.head(5)

(2799, 6)


,tweet_id,handle,retweet_count,text,mined_at,created_at
0,1358897168396992515,Kamala Harris,2136,RT @VP: Today @POTUS and I did a virtual tour ...,2021-02-09 03:06:53.048414,2021-02-08 21:55:06
1,1358851437942472704,Kamala Harris,9469,RT @POTUS: Making vaccines free and accessible...,2021-02-09 03:06:53.048431,2021-02-08 18:53:23
2,1358833875133284353,Kamala Harris,4028,RT @VP: The American Rescue Plan will get the ...,2021-02-09 03:06:53.048434,2021-02-08 17:43:36
3,1358538203096088576,Kamala Harris,34689,"RT @POTUS: For your family, for your neighbors...",2021-02-09 03:06:53.048436,2021-02-07 22:08:42
4,1358162630217789441,Kamala Harris,8697,RT @VP: It’s been a busy few weeks. Ready for ...,2021-02-09 03:06:53.048438,2021-02-06 21:16:19


In [9]:

tweets = pd.concat([biden_df, kamala_df], axis=0)
tweets.shape

(5599, 6)

# ***Pulling any interesting ngrams ***

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

In [11]:
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')
summaries = "".join(biden_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)
Counter(ngrams_summaries).most_common(20)

[('donald trump', 344),
 ('covid 19', 182),
 ('president trump', 128),
 ('american people', 98),
 ('health care', 89),
 ('vote https', 73),
 ('white house', 73),
 ('head https', 68),
 ('make sure', 58),
 ('build better', 56),
 ('today https', 53),
 ('president ll', 50),
 ('climate change', 49),
 ('election day', 49),
 ('need president', 48),
 ('united states', 47),
 ('president https', 47),
 ('need help', 44),
 ('ve got', 44),
 ('vote early', 38)]

In [12]:
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')
summaries = "".join(kamala_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)
Counter(ngrams_summaries).most_common(20)

[('rt joebiden', 115),
 ('donald trump', 101),
 ('health care', 74),
 ('covid 19', 73),
 ('supreme court', 68),
 ('american people', 60),
 ('united states', 51),
 ('make sure', 50),
 ('trump administration', 47),
 ('care act', 46),
 ('white house', 44),
 ('wear mask', 41),
 ('affordable care', 40),
 ('affordable care act', 40),
 ('biden harris', 37),
 ('years ago', 37),
 ('elect joebiden', 35),
 ('breonna taylor', 35),
 ('black women', 34),
 ('election day', 34)]

### **Tweets preprocessing and developing a model**
To classify which tweets corresponding to whoe, the below steps are required:

1.   Vectorizing the input text
2.   Initializing the model
3.   Hyperparameter tuning with Grid Searching
4.   Model optimization
5.   Model performance evaluation



In [13]:
tweet_text = tweets['text'].values.tolist()
#creating target
y = tweets['handle'].map(lambda x: 1 if x == 'Joe Biden' else 0).values
print(y)


[1 1 1 ... 0 0 0]


In [14]:
#Vectorizing with TF-IDF Vectorizer and creating X matrix
tfv = TfidfVectorizer(ngram_range=(2,4), max_features=2000)
X = tfv.fit_transform(tweet_text).todense()
print (X.shape)

(5599, 2000)


# **We will use only SVC model for this project**

In [15]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


In [16]:
params = {'kernel': ('linear', 'rbf'),'C': [1, 10]}
svr = svm.SVC()
#Grid searching to find optimal parameters for SVC Regression
clf = GridSearchCV(svr, params)
clf.fit(X, y)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [17]:
print (clf.best_params_)
print (clf.best_score_)

{'C': 10, 'kernel': 'rbf'}
0.7649567534788714


# **The below code might take several mintues so have a good cup of coffee or snackes you like**

In [18]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(svm.SVC(probability=True), X, y)

print (accuracies.mean())
print (1-y.mean())

0.7601381973701008
0.49991069833898916


# **Prediciton of probability of Biden & Kamala Tweets**

# **Do not re-initialize the tfidf vectorizer otherwise the features will be overwritten and the transform will not match the number of feature with your trained model. We only need to transform since we already fit it before. **

In [19]:
estimator = svm.SVC(C=10, kernel='linear',probability=True)
estimator.fit(X,y)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [20]:
# Prepare the tweets as TfIdf vectors
source_test = [
   "Extending the moratorium on evictions and foreclosures to help countless Americans keep a roof over their head — that’s the American Rescue Plan.",
    "My abiding hope—my abiding prayer—is that we emerge from this ordeal with a new wisdom to cherish simple moments to imagine new possibilities and to open our hearts just a little more to one another."]

# I copied the these two comments from Joe Biden and Kamala Harris twitter accounts correspondingly

Xtest = tfv.transform(source_test).todense()

In [21]:
pd.DataFrame(estimator.predict_proba(Xtest),columns=["Proba_Biden", "Proba_Kamala"])

,Proba_Biden,Proba_Kamala
0,0.604586,0.395414
1,0.330000,0.670000


Based on the model the first tweet came from Biden is around 70% probable and the probability of 2nd comment coming from Kamala is 72%

# **Now I am going to pull out the tweets both from Joe Biden and Kamala Harris with the highest and lowest probability of being Bien or Kamala based on the model we developed above**

In [22]:
proba=pd.DataFrame(estimator.predict_proba(X),columns=["Proba_Biden", "Proba_Kamala"])

In [23]:
joined_x = pd.merge(tweets, proba, left_index=True, right_index=True)
joined_x

,tweet_id,handle,retweet_count,text,mined_at,created_at,Proba_Biden,Proba_Kamala
0,1358851337522470914,Joe Biden,9469,RT @POTUS: Making vaccines free and accessible...,2021-02-09 03:06:46.924098,2021-02-08 18:52:59,0.063154,0.936846
0,1358897168396992515,Kamala Harris,2136,RT @VP: Today @POTUS and I did a virtual tour ...,2021-02-09 03:06:53.048414,2021-02-08 21:55:06,0.063154,0.936846
1,1358781635240599554,Joe Biden,11383,RT @POTUS: We don’t have a second to waste whe...,2021-02-09 03:06:46.924114,2021-02-08 14:16:01,0.083131,0.916869
1,1358851437942472704,Kamala Harris,9469,RT @POTUS: Making vaccines free and accessible...,2021-02-09 03:06:53.048431,2021-02-08 18:53:23,0.083131,0.916869
2,1358538143318880258,Joe Biden,34689,"RT @POTUS: For your family, for your neighbors...",2021-02-09 03:06:46.924117,2021-02-07 22:08:28,0.162862,0.837138
...,...,...,...,...,...,...,...,...
2797,1247676278938853376,Joe Biden,1722,"Folks, I know these are tough times, but this ...",2021-02-09 03:06:52.585180,2020-04-08 00:03:00,0.166291,0.833709
2797,1218349035666509824,Kamala Harris,3884,The responsibility to uphold the Constitution ...,2021-02-09 03:06:58.999639,2020-01-18 01:47:00,0.166291,0.833709
2798,1247659115683479555,Joe Biden,81,@blackenterprise Jill and I send our heartfelt...,2021-02-09 03:06:52.585182,2020-04-07 22:54:48,0.295245,0.704755
2798,1218294931686555648,Kamala Harris,6420,It would be wholly inappropriate for the Senat...,2021-02-09 03:06:58.999641,2020-01-17 22:12:01,0.295245,0.704755


In [29]:
joined_Biden = joined_x[joined_x['handle']=="Joe Biden"]
for el in joined_Biden[joined_Biden['Proba_Biden']==max(joined_Biden['Proba_Biden'])]['text']:
    print (el)

Supreme Court decisions affect our everyday lives, and the Constitution was designed to give voters a voice on who makes those decisions.

The Senate shouldn't act until after the American people select their next president and the next Congress.

Americans deserve to be heard.


In [30]:
for el in joined_Biden[joined_Biden['Proba_Biden']==min(joined_Biden['Proba_Biden'])]['text']:
    print (el)

RT @KamalaHarris: Donald Trump paid $750 in taxes. When I first heard about it, I literally said, “You mean $750,000?”


In [32]:
joined_Kamala = joined_x[joined_x['handle']=="Kamala Harris"]
for el in joined_Kamala[joined_Kamala['Proba_Kamala']==max(joined_Kamala['Proba_Kamala'])]['text']:
    print (el)

Donald Trump knew coronavirus was deadly, yet he still brought thousands of people together for his rallies—and wasn’t concerned about whether they were wearing masks or socially distanced. https://t.co/Wilg6O1bnq


In [33]:
for el in joined_Kamala[joined_Kamala['Proba_Kamala']==min(joined_Kamala['Proba_Kamala'])]['text']:
    print (el)

RT @JoeBiden: When President Trump took office, he inherited a growing economy from the Obama-Biden administration. And just like everythin…
